In [ ]:
%%html
<style type="text/css">
  span.ecb { background: yellow; }
</style>

# ATMO 5331 - Homework 3 - Fall 2025
## Due Sunday 12 Oct, 2025, 11:59 pm

When doing this homework, remember that you have two jobs:
1. Make it work.
2. Clean it up so that I can understand what you've done. If you think I might not understand, document it with a comment or a function docstring.

You should present your work with a clear logical progression. If that seems like a hassle, remember that in doing so you are practicing skills that are expected in your thesis and journal publications.

For this assignment you may work alone or in pairs. I will not be adjudicating relative level of effort in group work, so you are responsible for ensuring that you and your partner contribute equally.

<span class="ecb">Comments by ECB</span>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

For this assignment, download some [sample Ka data](https://www.atmo.ttu.edu/bruning/5331/Ka2140614021408.RAWPXA9.nc). These data were converted directly from the raw format collected by the radar (Sigmet) to the [NetCDF CF/Radial standard](http://ncar.github.io/CfRadial/). No further processing has been applied. You do not have to include this file in your repository.

In the file, the radar variables (such as reflectivity) are stored in arrays with dimension `(time, range)`. At each time, a single ray of data is collected, which extends along range. There is also a `sweep` dimension that corresponds to a few different variables. These variables mark the start and the end of each radar scan. Let's look at that structure:

In [ ]:
ds = xr.open_dataset('/data/KaSample/Ka2140614021408.RAWPXA9.nc', decode_timedelta=True)

print(ds)

# Print the first ten elevation angles.
print(ds.variables['elevation'][0:10])

# Shortcut to variable access
print(ds.elevation[0:10])

In [ ]:
print(ds.sweep_mode)
print(ds.sweep_start_ray_index)
print(ds.sweep_end_ray_index)

az = ds.azimuth[:]
el = ds.elevation[:]
t = ds.time[:]
fig, axs = plt.subplots(3,1, squeeze=False, figsize=(10,10))

axs[0,0].plot(t,el, '.k')
axs[0,0].set_ylabel('Elevation angle (deg)')
axs[0,0].set_xlabel('Time (UTC)')
axs[1,0].plot(t,az, '.k')
axs[1,0].set_ylabel('Azimuth angle (deg)')
axs[1,0].set_xlabel('Time (UTC)')
axs[2,0].plot(t, np.arange(t.shape[0]), '.k')
axs[2,0].set_ylabel('Index along time dimension')
axs[2,0].set_xlabel('Time (UTC)')
for sw_start, sw_end in zip(ds.sweep_start_ray_index, ds.sweep_end_ray_index):
    axs[0,0].plot(t[sw_start], el[sw_start], marker='o', color='g')
    axs[0,0].plot(t[sw_end], el[sw_end], marker='o',color='r')
    axs[2,0].plot(t[sw_start], sw_start, marker='o', color='g')
    axs[2,0].plot(t[sw_end], sw_end, marker='o',color='r')

The green dots mark the start of a sweep according to `sweep_start_ray_index`, and red marks the end according to `sweep_end_ray_index`. We observe that:

0. the scans are in [RHI](http://ww2010.atmos.uiuc.edu/(Gh)/guides/rs/rad/basics/cnmods.rxml) mode, per the scan_mode variable.
1. the data have time precision of 1 s
2. the data are not stored in time order.
3. the data are stored in the file in order of increasing elevation angle, not increasing time.
4. azimuth angle is constant.

For the rest of this assignment, let's work with the second of the five scans, where time and elevation both increase with index along the time dimension. Reuse this variable instead of hard-coding a scan number in later code.

In [ ]:
scan_idx = 1

**1.** Let's start by preparing the coordinates needed to plot a scan. Write a function that takes a radar dataset and scan index, and returns  2D arrays of ranges, azimuths, and elevations that correspond to the edges of each radar sample (a.k.a., each gate). Your function should return three separate 2D variables, as you see in the sample function call below. `coords_2d` should call another helper function, `spherical_coord_edges(d, scan_idx)` that returns the 1D coordinates along each coordinate direction for that sweep. I found it helpful to visualize how the coordinates were distributed in 2D a quick, unlableled plot of each coordinate: `plt.imshow(r)`.

Also print out the shapes of r, az, and el.

In [ ]:
def coords_2d(ds, scan_idx):
    # These are the wrong return values.
    return "r", "az", "el"


r, az, el = coords_2d(ds, scan_idx)

**2.** Make a 4-panel plot of reflectivity, velocity, spectrum width, and normalized coherent power. Use `coordinateSystems.RadarCoordinateSystem` together with a tangent plane system to plot in altitude above vs. range along a tangent plane. The tangent plane should be centered at the location of radar. You will need to definte a new variable that gives range along the tangent plane.

Note that if you find that the coordinate transformations fail to preserve the shape of your input coordinate arrays, you can fix it with `X.shape = r.shape`, where `X` has been transformed from `r`.

**3.** Repeat the plot for question 2, but for an aziuthal equidistant map projection centered at the radar location.

As above, you will to definte a new variable that gives a horizontal distance (a sort of "range") using the coordinates returned from the map projection. This can be calculated in one line of code.

How does this plot compare to the plot defined in tangent plane coordinates?

In [ ]:
# Where is the radar? What is its gate spacing?
print(ds.latitude)
print(ds.longitude)
print(ds.range[1]-ds.range[0])

To set the stage for the next two assignments, let's say we want to oversample the 15 m range gates to a 5 m carteisian grid in the domain 9.0…9.5 km range and 2.9…3.4 km altitude. Once again, we will use the distance above and the range along the tangent plane.

**4.** Create 2D arrays giving the corresponding center locations for each desired analysis location. Start with 1D grid box edges spanning the range above.

**5.** For the next question, we will interpolate in three ways:
- nearest neighbor
- linear
- the traditional meteorological "Barnes" analysis (which is really a Gaussian blur fliter). 

To do so, we will use MetPy's built-in interpolators. It wraps SciPy's built-in nearest and linear interpolation methods, and adds Barnes, natural neighbor, and other interpolators. 

You will probably need to install metpy with `conda install -c conda-forge metpy`.

Once you have installed MetPy, use its `interpolate_to_points` function. Note that it takes three arguments, and those arguments require that you use `np.vstack` to rearrange the locations and data into the correct shape. Where necessary you can use `flatten` to produce the sequence of 1D coordinate arrays accepted by `vstack`.

So that the Barnes analysis doesn't take forever, you will want to subset the data to only those locations that overlap with the analysis grid.

In [ ]:
# from scipy.interpolate import LinearNDInterpolator, NearestNDInterpolator, griddata
# from metpy.units import units
from metpy.interpolate import interpolate_to_points
interpolate_to_points?

**6.** Make a plot of the original data and the three interpolation methods. Compared to the original data, what do the three interpolation methods do to the data? Think about what you would regard as desirable properties of an analysis.

**Some demonstration code** showing some useful array tricks is provided below.

In [ ]:
import numpy as np

In [ ]:
np.vstack?

In [ ]:
a = np.array([1,2,3])
b = np.array([3,4,5])

In [ ]:
a

In [ ]:
b

In [ ]:
s=np.vstack((a,b))

In [ ]:
s

In [ ]:
s.shape

In [ ]:
s.T

In [ ]:
z=np.ones_like(s.T)

In [ ]:
z.flatten()

In [ ]:
np.hstack((a,b))

In [ ]:
z.shape

In [ ]:
np.hstack((z[:,0], z[:,1]))

In [ ]:
a[None,:].shape

In [ ]:
b[:,None].shape

In [ ]:
sq = a[None,:]*b[:,None]
sq

In [ ]:
sel = (sq > 5) & (sq <=10)
sel

In [ ]:
np.argmin(sq)

In [ ]:
np.where(sel)

In [ ]:
np.where?

In [ ]:
ds

In [ ]:
r_cond = (ds.range > 5000) & (ds.range < 10000)
sweep_cond = ((ds.sweep >= 2) & (ds.sweep <=4))

ds[ {'range':r_cond, 'sweep':sweep_cond} ]